### Raspagem de Dados

A célula abaixo é responsável em receber o nome de um fundo e pesquisar no site fundExplorer.com.br a cotação e dividendo atual.

In [1]:
import requests
from bs4 import BeautifulSoup


def valor_atual(fii):
    valor = 0
    dividendo = 0
    fii = fii.upper()
    link_fii = 'http://www.fundsexplorer.com.br/funds/'+fii
    try:
        page = requests.get(link_fii)
        soup = BeautifulSoup(page.content, 'html.parser')
        p = soup.findAll('div', class_='right')
        valor = p[0].find('div', class_='count').contents[0]
        valor = str(valor)
        valor = valor.replace(',','.')
        valor = float(valor)
        dividendo = p[2].find('div', class_='count').contents[0]
        dividendo = str(dividendo)
        dividendo = dividendo.replace(',','.')
        dividendo = float(dividendo)
    except:
        print("Erro na raspagem de dados do fundo: " + fii)
    
    return valor, dividendo

### Aplicação da Programação Linear

A célula abaixo é responsável em receber uma lista de Fundos Imobiliários e o valor a ser aplicado.
De posse destes dados, calcula a maximização das alocações, baseada no Yeld.

In [2]:
import pulp

def alocar_recurso(fii_listados, quantidade_aplicado):
    lista_cotacao = []
    lista_variaveis = []
    lista_dividendos = []

    print(40 * "*")
    print(" Cotação do dia ")
    for fundo in fii_listados:
        valor_e_dividendo = valor_atual(fundo)
        if valor_e_dividendo[0] > 0:
            lista_variaveis.append(pulp.LpVariable(fundo, lowBound=0, cat='Integer'))
            lista_cotacao.append(valor_e_dividendo[0])
            lista_dividendos.append(valor_e_dividendo[1])
            print("FII " + str(fundo) + " cotado a R$ " + str(valor_e_dividendo[0]) )
            print("último dividendo de " + str(fundo) + " =  R$ " + str(valor_e_dividendo[1]) + "\n")
    print(40 * "*")

    model = pulp.LpProblem("Maximizar ganhos baseado nos dividentos ", pulp.LpMaximize)

    maximizar = ""
    for fundo in range(len(lista_variaveis)):
        maximizar += lista_variaveis[fundo] * lista_dividendos[fundo]
    model += maximizar

    restricao = ""
    for fundo in range(len(lista_variaveis)):
        restricao += lista_variaveis[fundo] * lista_cotacao[fundo]
    model += restricao <= quantidade_aplicado, 'Maximo que posso aplicar'

    # print(model)

    # Resolve o problema
    model.solve()
        
    if pulp.LpStatus[model.status] == "Optimal":
        print("A solução existe e foi encontrada")
        print ("Rendimento baseado nos dividendos: R$ " + str(pulp.value(model.objective)))
        return lista_variaveis

    if pulp.LpStatus[model.status] == "Not Solved":
        print("Solução não encontrada")
        return []
    
    if pulp.LpStatus[model.status] == "INFEASIBLE":
        print("O problema não tem solução viável.")
        return []


### Simulação de aplicações

Altere o <b>valor_para_aplica </b> e a <b>lista de fundos</b> que deseja adicionar em sua carteira.

In [6]:
valor_para_aplicar = 500

fundos = []
fundos.append('hglg11')
fundos.append('ctxt11')
fundos.append('abcp11')
fundos.append('mfii11')

maximizado = alocar_recurso(fundos, valor_para_aplicar)

if len(maximizado) > 0:
    print("\n")
    print(40 * "*")
    print("Como devo aplicar")
    for fundo in maximizado:
        print("     Fundo Imobiliário " + str(fundo) + " =  " + str(fundo.varValue) + " cotas ")


****************************************
 Cotação do dia 
FII hglg11 cotado a R$ 135.83
último dividendo de hglg11 =  R$ 0.75

FII ctxt11 cotado a R$ 4.07
último dividendo de ctxt11 =  R$ 0.0274

FII abcp11 cotado a R$ 16.65
último dividendo de abcp11 =  R$ 0.085

FII mfii11 cotado a R$ 98.9
último dividendo de mfii11 =  R$ 1.13

****************************************
A solução existe e foi encontrada
Rendimento baseado nos dividendos: R$ 5.6774


****************************************
Como devo aplicar
     Fundo Imobiliário hglg11 =  0.0 cotas 
     Fundo Imobiliário ctxt11 =  1.0 cotas 
     Fundo Imobiliário abcp11 =  0.0 cotas 
     Fundo Imobiliário mfii11 =  5.0 cotas 
